In [1]:
import os
import sys
import math
import logging
from pathlib import Path

import numpy as np
import scipy as sp
import sklearn
import statsmodels.api as sm
from statsmodels.formula.api import ols

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from enadepy.loaders import read_interm
from enadepy.helpers import list_cols_socioecon
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

In this pre-processing step, we use a previously filtered data containing only registries for students from public institutions.

Here, we will select variables of interest by hand, as an initial automated approach did not show very interesting results.

Related notebooks:

    - 3.0-mc-preprocess-pub
    - 4.0-mc-mining-pub

In [3]:
df = read_interm('../data/interim/microdados_enade_ata_2016a2018_pub.csv')

In [4]:
df.head()

,NU_ANO,NT_GER,CO_IES,CO_GRUPO,NU_IDADE,TP_SEXO,ANO_FIM_EM,ANO_IN_GRAD,QE_I01,QE_I02,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,QE_I10,QE_I11,QE_I12,QE_I13,QE_I14,QE_I15,QE_I17,QE_I18,QE_I19,QE_I20,QE_I21,QE_I22,QE_I23,QE_I24,QE_I25
0,2016,52.9,56,6,26,M,2007,2011,A,A,B,B,E,A,A,B,A,A,B,F,A,A,E,A,B,K,A,B,C,D,A
1,2016,74.4,56,6,23,M,2010,2012,A,A,D,D,B,C,B,B,A,A,A,A,A,A,B,A,B,C,A,C,C,A,E
2,2016,60.2,56,6,26,F,2007,2010,A,A,E,E,D,D,D,B,A,A,A,A,A,A,B,A,B,E,A,B,D,E,H
3,2016,70.6,56,6,25,F,2008,2012,A,A,E,D,D,A,D,B,A,A,A,A,A,A,B,A,B,C,A,C,B,E,B
4,2016,30.9,56,6,25,F,2008,2011,A,A,B,B,A,A,B,B,A,A,A,C,A,A,A,A,B,C,A,B,C,E,E


In [5]:
df.shape

(254, 31)

In [6]:
socio_cols = [col for col in df.columns if col.startswith('QE_')]

See all questions from the socioeconomics questionnaire and their results

In [7]:
dfmelt = df[socio_cols].melt(var_name="Question", value_name="Choice")

In [8]:
pd.crosstab(index=dfmelt['Question'], columns=dfmelt['Choice'])

Choice,A,B,C,D,E,F,G,H,I,J,K
Question,,,,,,,,,,,
QE_I01,209,34,7,0,4,0,0,0,0,0,0
QE_I02,193,9,10,40,0,2,0,0,0,0,0
QE_I04,5,38,30,89,71,21,0,0,0,0,0
QE_I05,3,30,35,85,69,32,0,0,0,0,0
QE_I06,30,116,42,56,8,2,0,0,0,0,0
QE_I07,58,28,57,71,19,17,4,0,0,0,0
QE_I08,21,49,55,51,49,26,3,0,0,0,0
QE_I09,10,162,56,13,11,2,0,0,0,0,0
QE_I10,182,15,11,19,27,0,0,0,0,0,0


In [9]:
socio_sel_cols = ['QE_I02', 'QE_I04', 'QE_I05', 'QE_I06', 'QE_I07', 'QE_I08', 'QE_I09', 'QE_I17', 'QE_I22', 'QE_I23', 'QE_I25']

In [10]:
cols = socio_sel_cols + ['TP_SEXO', 'NT_GER']

In [11]:
cols

['QE_I02',
 'QE_I04',
 'QE_I05',
 'QE_I06',
 'QE_I07',
 'QE_I08',
 'QE_I09',
 'QE_I17',
 'QE_I22',
 'QE_I23',
 'QE_I25',
 'TP_SEXO',
 'NT_GER']

### Split data according to skin color

This split is necessary because there is a dominance of white students in the dataset. By splitting the data, we can look for patterns when students are not  white.

In [12]:
df.QE_I02.value_counts()

A    193
D     40
C     10
B      9
F      2
Name: QE_I02, dtype: Int64

In [13]:
df_white = df.query('QE_I02 == "A"')[cols].drop(columns="QE_I02")
df_white

,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,QE_I17,QE_I22,QE_I23,QE_I25,TP_SEXO,NT_GER
0,B,B,E,A,A,B,E,B,C,A,M,52.9
1,D,D,B,C,B,B,B,C,C,E,M,74.4
2,E,E,D,D,D,B,B,B,D,H,F,60.2
3,E,D,D,A,D,B,B,C,B,B,F,70.6
4,B,B,A,A,B,B,A,B,C,E,F,30.9
...,...,...,...,...,...,...,...,...,...,...,...,...
249,E,D,C,B,B,E,A,B,D,B,F,53.9
250,C,B,B,D,C,B,A,C,C,H,F,49.2
251,C,D,B,D,C,A,A,B,D,E,M,41.3
252,D,D,B,G,D,B,B,A,B,H,F,59.1


In [14]:
df_black = df.query('QE_I02 in ["B", "D"]')[cols].drop(columns="QE_I02")
df_black

,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,QE_I17,QE_I22,QE_I23,QE_I25,TP_SEXO,NT_GER
9,C,D,D,A,D,B,B,A,B,D,F,67.2
13,E,E,E,B,A,A,B,C,C,A,F,61.1
15,E,E,D,A,E,B,B,B,B,E,M,51.7
38,D,B,F,C,B,B,A,A,B,A,F,60.4
48,D,E,C,B,E,B,B,C,B,E,F,63.0
54,F,F,D,A,E,C,B,A,D,E,F,75.4
69,E,E,D,A,E,B,F,E,E,H,F,59.9
82,E,D,D,A,E,F,B,C,B,H,M,39.2
87,D,D,D,C,C,B,B,B,B,A,M,70.6
102,F,E,D,A,D,B,A,C,C,H,F,54.2


## Transforming data

In [15]:
for df_final, sufix in zip([df_white, df_black], ["white", "nowhite"]):
    df_final['NT_GER'] = pd.qcut(df_final.NT_GER, 2, labels=["Q0", "Q1"])
    cats = df_final.columns
    enc = preprocessing.OneHotEncoder(sparse=False)
    # fit and transform in one call and print categories
    out_enc = enc.fit_transform(df_final[cats])
    new_cols = enc.get_feature_names(cats).tolist()
    print(new_cols)
    # create temp. dataframe for concatenation with original data
    df_enc = pd.DataFrame(data=out_enc, columns=new_cols)
    df_enc.index = df_final.index
    # drop original columns and concatenate new encoded columns
    df_final.drop(cats, axis=1, inplace=True)
    df_final = pd.concat([df_final, df_enc], axis=1)
    print(df_final.columns)
    df_final.to_csv(f'../data/preprocessed/enade_2016a2018_pub_onehot_{sufix}.csv', index=False)

['QE_I04_A', 'QE_I04_B', 'QE_I04_C', 'QE_I04_D', 'QE_I04_E', 'QE_I04_F', 'QE_I05_A', 'QE_I05_B', 'QE_I05_C', 'QE_I05_D', 'QE_I05_E', 'QE_I05_F', 'QE_I06_A', 'QE_I06_B', 'QE_I06_C', 'QE_I06_D', 'QE_I06_E', 'QE_I06_F', 'QE_I07_A', 'QE_I07_B', 'QE_I07_C', 'QE_I07_D', 'QE_I07_E', 'QE_I07_F', 'QE_I07_G', 'QE_I08_A', 'QE_I08_B', 'QE_I08_C', 'QE_I08_D', 'QE_I08_E', 'QE_I08_F', 'QE_I08_G', 'QE_I09_A', 'QE_I09_B', 'QE_I09_C', 'QE_I09_D', 'QE_I09_E', 'QE_I17_A', 'QE_I17_B', 'QE_I17_C', 'QE_I17_D', 'QE_I17_E', 'QE_I17_F', 'QE_I22_A', 'QE_I22_B', 'QE_I22_C', 'QE_I22_D', 'QE_I22_E', 'QE_I23_A', 'QE_I23_B', 'QE_I23_C', 'QE_I23_D', 'QE_I23_E', 'QE_I25_A', 'QE_I25_B', 'QE_I25_C', 'QE_I25_D', 'QE_I25_E', 'QE_I25_G', 'QE_I25_H', 'TP_SEXO_F', 'TP_SEXO_M', 'NT_GER_Q0', 'NT_GER_Q1']
Index(['QE_I04_A', 'QE_I04_B', 'QE_I04_C', 'QE_I04_D', 'QE_I04_E', 'QE_I04_F',
       'QE_I05_A', 'QE_I05_B', 'QE_I05_C', 'QE_I05_D', 'QE_I05_E', 'QE_I05_F',
       'QE_I06_A', 'QE_I06_B', 'QE_I06_C', 'QE_I06_D', 'QE_I06_E', 'Q